# Init

In [1]:
### Standard ###
import sys, os

### Special ###
import numpy as np

### Local ###
sys.path.append( "../pddlstream/" )
from pddlstream.algorithms.meta import solve, create_parser
from pddlstream.algorithms.search import solve_from_pddl
from pddlstream.utils import read
from pddlstream.language.constants import print_solution, PDDLProblem
from pddlstream.utils import read, get_file_path, Profiler
from pddlstream.language.constants import PDDLProblem, And, Equal, print_solution
from examples.discrete_belief.run import revisit_mdp_cost, MAX_COST, clip_cost

# Streams

In [2]:
class Pose:
    """ A named object, it's pose, and the surface that supports it """
    
    def __init__( self, name, pose, surf ):
        self.name = name
        self.pose = pose
        self.surf = surf # WARNING: I do not like that this is part of the predicate!

    def iterate( self ):
        yield self


def get_pose_stream( robot, camera, detector ):
    """ ??? Return a function that returns poses ??? """
    # NOTE: ??? This function assumes that the camera, detector, and robot were initialized ???

    blockNames  = ['redBlock', 'ylwBlock', 'bluBlock',]

    def rtn_func():
        """ ??? A function that returns poses ??? """
        pcd, rgbdImage = detector.real.get_PCD()
        depthImage, colorImage = rgbdImage.depth, rgbdImage.color
        urPose = robot.get_tcp_pose()
        blocks = detector.getBlocksFromImages( colorImage, depthImage, urPose, display = False )

        print( "!HEY!, The pose stream was evaluated!" )

        while True: # WARNING: I DO NOT KNOW WHY THIS PREVENTS REDUNDANT STATES
            for i, blockName in enumerate( blockNames ):
                pose = blocks[i].worldFrameCoords
                # WARNING: I DO NOT KNOW WHY THIS IS A `yield` STATEMENT
                # yield Pose( blockName, pose, 'table' ) # WARNING: HACK, The supporting object is hardcoded
                yield (Pose( blockName, pose, 'table' ),) # WARNING: HACK, The supporting object is hardcoded

    return rtn_func

# Hardware + Perception

In [3]:
sys.path.append( "../" )
from magpie.ur5 import UR5_Interface
from Perception import DepthCam, ObjectDetection

def magpie_init():
    """ Start MAGPIE-related hardware """
    # QUESTION: SHOULD THERE BE A SINGLETON MAGPIE "MANAGER" OBJECT?
    robot = UR5_Interface()
    robot.start()
    camera = DepthCam()
    detector = ObjectDetection( camera, None, moveRelative = True) # FIXME: I HATE THIS ENTANGLEMENT
    return robot, camera, detector

def magpie_shutdown( robot, camera ):
    """ Start MAGPIE-related hardware """
    robot.stop()
    camera.stop()

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [4]:
robot, camera, detector = magpie_init()

Succeeded to open the port
Succeeded to change the baudrate
Moving speed of dxl ID: 1 set to 100 
Moving speed of dxl ID: 2 set to 100 
Depth stream started!
Color stream started!


# PDDLStream Context

In [5]:
domain_pddl  = read( get_file_path( os.path.join(os.getcwd(), '40_blocks_detect'), 'domain.pddl' ) )
stream_pddl  = read( get_file_path( os.path.join(os.getcwd(), '40_blocks_detect'), 'stream.pddl' ) )
constant_map = {}

In [6]:
scan_cost = 1
init      = [
    Equal(('MoveCost',), 1),
    Equal(('PickCost',), 1),
    Equal(('PlaceCost',), 1),
    Equal(('ScanCost',), scan_cost),
]

In [7]:
blockNames  = ['redBlock', 'ylwBlock', 'bluBlock',]
supportName = 'table'

init += [
    ('Observable', np.eye(4)),
]

for blockName in blockNames:
    pose = np.eye(4)
    init += [
        ('Pose'  , blockName, pose), 
        ('AtPose', blockName, pose),
        ('Scannable'   , blockName),
        ('Registerable', blockName),
        ('Graspable'   , blockName),
        ('Uncertain'   , blockName),
    ]

goal = ('and',
    ('Localized', 'redBlock'),
    ('Localized', 'ylwBlock'),
    ('Localized', 'bluBlock'),
    # ('on', 'ylwBlock', 'bluBlock',),
    # ('on', 'redBlock', 'ylwBlock',),
)

stream_map = {
    'sample-pose': get_pose_stream( robot, camera, detector ), # Enclose MAGPIE connections in a stream
}

print( stream_map['sample-pose'] )

problem = PDDLProblem( domain_pddl, constant_map, stream_pddl, stream_map, init, goal )

print(  dir( problem )  )

print(  problem.goal  )
print(  problem.stream_map  )

<function get_pose_stream.<locals>.rtn_func at 0x7fca9056b4c0>
['__add__', '__class__', '__class_getitem__', '__contains__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getnewargs__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__mul__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__rmul__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '_asdict', '_field_defaults', '_fields', '_make', '_replace', 'constant_map', 'count', 'domain_pddl', 'goal', 'index', 'init', 'stream_map', 'stream_pddl']
('and', ('Localized', 'redBlock'), ('Localized', 'ylwBlock'), ('Localized', 'bluBlock'))
{'sample-pose': <function get_pose_stream.<locals>.rtn_func at 0x7fca9056b4c0>}


# Evaluate Stream(s)

In [8]:
# class SolverArgs:
#     """ PDDL Solver Args """
#     def __init__( self, algo, unit ):
#         self.algorithm = algo
#         self.unit      = unit

# args = SolverArgs(  )

parser = create_parser()
parser.add_argument('-f', '--foo', type=str, required=False, help='UNK') # REQUIRED for Jupyter
args = parser.parse_args()
print( 'Arguments:', args )

Arguments: Namespace(algorithm='adaptive', unit=False, foo='/home/james/.local/share/jupyter/runtime/kernel-64eadb6e-ae10-4e20-a692-fb9fdfbe785b.json')


In [9]:
from pprint import pprint

stream_info = {
    # 'test-vis-base': StreamInfo(eager=True, p_success=0),
    # 'test-reg-base': StreamInfo(eager=True, p_success=0),
}
hierarchy = [
    # ABSTRIPSLayer(pos_pre=['AtBConf']),
]
verbose = True

# solution = solve( problem, algorithm=args.algorithm, unit_costs=args.unit,
solution = solve( problem, algorithm=args.algorithm, unit_costs=1,
                  # stream_info=stream_info, hierarchy=hierarchy, debug=False,
                  stream_info=stream_info, hierarchy=hierarchy, debug=True,
                  success_cost=MAX_COST, verbose=verbose)

print( dir( solution ) )
print()

for elem in solution.certificate[0]:
    pprint( elem )
    print()

Streams: [sample-pose:('?o', '?r')->('?p',)]
Functions: []
Negated: []
Optimizers: []

Iteration: 1 | Complexity: 0 | Skeletons: 0 | Skeleton Queue: 0 | Disabled: 0 | Evaluations: 23 | Eager Calls: 0 | Cost: inf | Search Time: 0.000 | Sample Time: 0.000 | Total Time: 0.000

Generating Datalog program... [0.000s CPU, 0.001s wall-clock]
Normalizing Datalog program...
Normalizing Datalog program: [0.000s CPU, 0.002s wall-clock]
Preparing model... [0.000s CPU, 0.001s wall-clock]
Generated 9 rules.
Computing model... [0.010s CPU, 0.001s wall-clock]
40 relevant atoms
0 auxiliary atoms
40 final queue length
40 total queue pushes
Completing instantiation... [0.000s CPU, 0.000s wall-clock]
Infeasible: True
Instantiation time: 0.007s
Attempt: 1 | Results: 0 | Depth: 0 | Success: False | Time: 0.008
Stream plan (inf, 0, inf): False
Action plan (inf, inf): False
Summary: {complexity: 0, cost: inf, evaluations: 23, iterations: 1, length: inf, run_time: 0.008, sample_time: 0.000, search_time: 0.008,

# Shutdown

In [10]:
magpie_shutdown( robot, camera )

RTDEControlInterface: RTDE control script is not running!
RTDEControlInterface: RTDE control script is not running!
